In [1]:
!nvidia-smi

No devices were found


# Data preparation
### Note: change the label_tag as you need (for different disease tag). 
#### --label_tag "LABEL_1" (~~~~~~~ from "LABEL_1" to "LABEL_14")

In [ ]:
!python /root/autodl-nas/CoAtNet_NIH/dataloader/data_prepare.py /root/autodl-nas/nih_database /root/autodl-nas/nih_database/Data_Entry_2017.csv --save_path /root/autodl-nas/CoAtNet_NIH/yamls/NIH_multi.yaml --file_tag "Image Index" --label_tag "Finding Labels" --multi_label

### Keep the datasets with fixed divisions. 

In [ ]:
!python /root/autodl-nas/CoAtNet_NIH/dataloader/data_prepare.py /root/autodl-nas/nih_database /root/autodl-nas/nih_database/Data_Entry_2017_train.csv --save_path /root/autodl-nas/CoAtNet_NIH/yamls/NIH_multi_train.yaml --file_tag "Image Index" --label_tag "Finding Labels" --multi_label
!python /root/autodl-nas/CoAtNet_NIH/dataloader/data_prepare.py /root/autodl-nas/nih_database /root/autodl-nas/nih_database/Data_Entry_2017_val.csv --save_path /root/autodl-nas/CoAtNet_NIH/yamls/NIH_multi_val.yaml --file_tag "Image Index" --label_tag "Finding Labels" --multi_label
!python /root/autodl-nas/CoAtNet_NIH/dataloader/data_prepare.py /root/autodl-nas/nih_database /root/autodl-nas/nih_database/Data_Entry_2017_test.csv --save_path /root/autodl-nas/CoAtNet_NIH/yamls/NIH_multi_test.yaml --file_tag "Image Index" --label_tag "Finding Labels" --multi_label

# Dataset division
### Train, val, test (0.7, 0.1, 0.2)

In [ ]:
!python /root/autodl-nas/CoAtNet_NIH/dataloader/data_split.py /root/autodl-nas/CoAtNet_NIH/yamls/NIH_multi.yaml --split 0.7 0.1 0.2

# Model training
### Before training, modify the NIH.yaml and train.py file as needed: 
#### 1. NIH.yaml: "train_path", "val_path", "log_path", and "save_path". 
#### 2. train.py: line155, default="/content/drive/MyDrive/CoAtNet_NIH/config/NIH.yaml".
#### 3. coatnet.py: line245, change the image size to what you need. 

In [ ]:
# Mean and SD
!python /root/autodl-nas/CoAtNet_NIH/dataloader/get_mean_std.py --data /root/autodl-nas/CoAtNet_NIH/yamls/NIH_multi_train.yaml --channel 1

In [ ]:
!torchrun --standalone --nproc_per_node=4 --nnodes=1 /root/autodl-nas/CoAtNet_NIH/train_distributed.py --config /root/autodl-nas/CoAtNet_NIH/config/NIH_multi_resnet50_C14.yaml --model ResNet_50
!torchrun --standalone --nproc_per_node=4 --nnodes=1 /root/autodl-nas/CoAtNet_NIH/train_distributed.py --config /root/autodl-nas/CoAtNet_NIH/config/NIH_multi_vit_b16_C14.yaml --model Vit_b16
!torchrun --standalone --nproc_per_node=4 --nnodes=1 /root/autodl-nas/CoAtNet_NIH/train_distributed.py --config /root/autodl-nas/CoAtNet_NIH/config/NIH_multi_efficient_b5_C14.yaml --model EfficientNet_b5
!torchrun --standalone --nproc_per_node=4 --nnodes=1 /root/autodl-nas/CoAtNet_NIH/train_distributed.py --config /root/autodl-nas/CoAtNet_NIH/config/NIH_multi_coatnet0_ln_C14.yaml --model CoAtNet_0_ln



# Model evaluation

In [ ]:
!python /root/autodl-nas/CoAtNet_NIH/evaluate_multi.py --config /root/autodl-nas/CoAtNet_NIH/config/NIH_multi_coatnet0_ln_C14.yaml --model_path /root/autodl-nas/CoAtNet_NIH/ckpt/CoAtNet_0_ln_NIH_multi_best_final.pt